### 微调前

In [2]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [3]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Request cae86327-1fd9-43ee-86bb-ac80b2c10ff9: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/config.json (authenticated: False)
Request 26f0167a-9201-4575-a29c-95e36c184047: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/adapter_config.json (authenticated: False)
Request 78cca1fd-8ed6-4841-b4a2-be34ebcea8b0: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/configuration_chatglm.py (authenticated: False)
Request b2db9cd6-0231-4fe9-a66c-20683026d515: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/modeling_chatglm.py (authenticated: False)
Request 4fbbdf91-4f0f-4d34-b56b-6771b5cfd896: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/main/adapter_config.json (authenticated: False)


Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

Request 827e6c9c-7fcb-4f31-812c-383b117c669e: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/generation_config.json (authenticated: False)


In [4]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

Request 4a91a8df-e909-4ede-b772-66ade2638ad6: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/tokenizer_config.json (authenticated: False)
Request 5700edfb-01aa-4ceb-9472-0adc3ce7d401: HEAD https://hf-mirror.com/THUDM/chatglm3-6b/resolve/b098244/tokenization_chatglm.py (authenticated: False)


In [6]:
input_text = "解释下乾卦是什么？"

In [7]:
response, history = base_model.chat(tokenizer, query=input_text)

In [8]:
print(response)

乾卦是八卦之一，也是八宫图之一，其卦象由三个阳爻夹一个阴爻构成，象征着天、云和雷。乾卦的卦辞是“元、亨、利、贞”，表示天圆、地圆、人生圆形，是一种最理想、最圆满的状态。乾卦的含义广泛，可以代表阳、力量、刚强、积极、正义等。在五行中，乾卦代表金，因此，乾卦也可以代表刚强、坚定、稳固、重义等。在命理学中，乾卦被认为是一种吉祥的符号，象征着好运、成功和幸福。


In [9]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是六十四卦中的第五卦。它的卦象由两个阴爻夹一个阳爻构成，象征着天上的云彩和地上的水，表示诉讼、争端、矛盾和 dispute。

讼卦的卦辞是“无言、元吉、亨”，表示虽然双方没有言语，但仍然可以获得吉祥，并且能够顺利地解决问题。从五行角度来看，讼卦属于水行，因此，它象征着柔、包容、适应和妥协。在命理学中，讼卦被认为是一种凶卦，表示可能会遇到困难和挫折，但只要能够采取适当的策略和态度，就能够化解矛盾，解决问题。

讼卦的意义不仅限于诉讼和争端，也适用于其他方面的矛盾和问题。它告诉我们，在面对问题和矛盾时，应该采取包容、适应和妥协的态度，以达到和谐和平解决矛盾的目的。


### QLoRA微调后

In [10]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240401_103955"

peft_model_path = f"/root/autodl-tmp//THUDM/chatglm3-6b-epoch3-20240401_103955"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [11]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

In [12]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，由两个乾卦叠加而成，代表着天。它象征着天象，以及与天相关的哲学观念。在周易中，乾卦代表刚健、健行、刚健不屈的意境。它预示着事情的进展将如君子行走在平地，柔顺如 flex 下的君子，而乾卦本身也代表着刚健、刚正、刚强、不屈不挠的精神。乾卦的核心哲学是：天行健，君子自强不息。乾卦的力量在动，而不是静止的，因此动卦如NAKASHIC。动卦如NAKASHIC，代表着动不动就行的意思，也预示着动不动就行的君子自强不息。在事业方面，动卦如NAKASHIC，预示着君子事业自强不息，不断求进，并且压倒一切困难，最终获得成功。在经商方面，动卦如NAKASHIC，预示着君子经商如自强不息，不断求进，并且能够取得成功。在求名方面，动卦如NAKASHIC，预示着君子求名如自强不息，不断求进，并且能够取得成功。在决策方面，动卦如NAKASHIC，提示自强不息的作风，必须不断求进，并且积极地解决问题，才能得到良策。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240401_103955）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [13]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240401_103955）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


In [14]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240401_103955）：
[gMASK]sop 师卦是什么？ 在周易中，师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。这一卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地中有水的情景，寓意着君子应当像大地一样容纳和畜养大众。师卦的解释强调选择德高望重的长者来统率军队，才能获得吉祥无咎。另外，师卦也象征着困难重重，需要包容别人、艰苦努力，及时行事，严于律已。在事业、经商、求名、婚恋等方面的决策中，都需要警惕潜在敌人，小心谨慎，合作与决断兼顾，方能成功。
